In [28]:
import csv
import json
import pandas as pd
import datetime as dt

csvfile = open('./data/restaurant-and-market-health-violations.csv', 'r')



pdCSV = pd.DataFrame.from_csv(csvfile,index_col=24)
pdCSV.head()
csvfile.close()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  # Remove the CWD from sys.path while we load stuff.


In [21]:
pdDate = pdCSV.loc[(pdCSV['activity_date'] > '2018-06-30')&(pdCSV['activity_date'] <='2018-07-31')]
pdDate.head(100)

,serial_number,activity_date,facility_name,violation_code,violation_description,violation_status,points,grade,facility_address,facility_city,...,owner_id,owner_name,pe_description,program_element_pe,program_name,program_status,record_id,score,service_code,service_description
row_id,,,,,,,,,,,,,,,,,,,,,
DA01KEPWCF045,DA01KEPWC,2018-07-18T00:00:00.000,LA FARFALA COFFE SHOP,F045,# 45. Sleeping quarters,OUT OF COMPLIANCE,1,A,346 N FORD BLVD,LOS ANGELES,...,OW0185212,DANERY S. VELASQUEZ,RESTAURANT (0-30) SEATS MODERATE RISK,1631,LA FARFALA COFFE SHOP,ACTIVE,PR0173373,93,1,ROUTINE INSPECTION
DA006DSLSF006,DA006DSLS,2018-07-26T00:00:00.000,GARCONS DE CAFE,F006,# 06. Adequate handwashing facilities supplied...,OUT OF COMPLIANCE,2,A,541 S SPRING ST # 114,LOS ANGELES,...,OW0245826,KC INVESTMENT LLC,RESTAURANT (0-30) SEATS MODERATE RISK,1631,GARCONS DE CAFE,ACTIVE,PR0201189,95,1,ROUTINE INSPECTION
DA006DSLSF040,DA006DSLS,2018-07-26T00:00:00.000,GARCONS DE CAFE,F040,"# 40. Plumbing: Plumbing in good repair, prope...",OUT OF COMPLIANCE,1,A,541 S SPRING ST # 114,LOS ANGELES,...,OW0245826,KC INVESTMENT LLC,RESTAURANT (0-30) SEATS MODERATE RISK,1631,GARCONS DE CAFE,ACTIVE,PR0201189,95,1,ROUTINE INSPECTION
DA006DSLSF052,DA006DSLS,2018-07-26T00:00:00.000,GARCONS DE CAFE,F052,# 01b. Food safety certification,OUT OF COMPLIANCE,2,A,541 S SPRING ST # 114,LOS ANGELES,...,OW0245826,KC INVESTMENT LLC,RESTAURANT (0-30) SEATS MODERATE RISK,1631,GARCONS DE CAFE,ACTIVE,PR0201189,95,1,ROUTINE INSPECTION
DA00GQVTQF008,DA00GQVTQ,2018-07-19T00:00:00.000,ACE SUSHI @ VONS,F008,# 08. Time as a public health control; procedu...,OUT OF COMPLIANCE,2,A,4030 S CENTINELA AVE,LOS ANGELES,...,OW0208935,ASIANA MANAGEMENT GR,RESTAURANT (0-30) SEATS HIGH RISK,1632,ACE SUSHI @ VONS,ACTIVE,PR0188606,96,1,ROUTINE INSPECTION
DA00GQVTQF014,DA00GQVTQ,2018-07-19T00:00:00.000,ACE SUSHI @ VONS,F014,# 14. Food contact surfaces: clean and sanitized,OUT OF COMPLIANCE,2,A,4030 S CENTINELA AVE,LOS ANGELES,...,OW0208935,ASIANA MANAGEMENT GR,RESTAURANT (0-30) SEATS HIGH RISK,1632,ACE SUSHI @ VONS,ACTIVE,PR0188606,96,1,ROUTINE INSPECTION
DA00RZ5GNF008,DA00RZ5GN,2018-07-24T00:00:00.000,PAPA JOHNS PIZZA,F008,# 08. Time as a public health control; procedu...,OUT OF COMPLIANCE,2,A,13350 WASHINGTON BLVD,LOS ANGELES,...,OW0128419,"PJCA-1, LP",RESTAURANT (0-30) SEATS MODERATE RISK,1631,PAPA JOHNS PIZZA,ACTIVE,PR0153550,96,1,ROUTINE INSPECTION
DA00RZ5GNF033,DA00RZ5GN,2018-07-24T00:00:00.000,PAPA JOHNS PIZZA,F033,# 33. Nonfood-contact surfaces clean and in go...,OUT OF COMPLIANCE,1,A,13350 WASHINGTON BLVD,LOS ANGELES,...,OW0128419,"PJCA-1, LP",RESTAURANT (0-30) SEATS MODERATE RISK,1631,PAPA JOHNS PIZZA,ACTIVE,PR0153550,96,1,ROUTINE INSPECTION
DA00RZ5GNF046,DA00RZ5GN,2018-07-24T00:00:00.000,PAPA JOHNS PIZZA,F046,# 46. Signs posted; last inspection report ava...,OUT OF COMPLIANCE,1,A,13350 WASHINGTON BLVD,LOS ANGELES,...,OW0128419,"PJCA-1, LP",RESTAURANT (0-30) SEATS MODERATE RISK,1631,PAPA JOHNS PIZZA,ACTIVE,PR0153550,96,1,ROUTINE INSPECTION


In [24]:
pdDate.to_json(r'./data/restaurant-and-market-health-violations.json', orient='records')